# Tutorial: How to wrap a C-Function in Cython

This simple tutorial is ment to explain in a simple way how to include a C-function into python using cython.
We explain which files are needed, what their format needs to be and which cython syntax to use to include the C-function into python. This Tutorial assumes basic knowledge of C and python and nearly no knowledge of cython.

We provide a minimal example finding the closest element to ...


## Needed Files

In order to wrap a C-function into python one needs 5 files:
- .h file with the function declaration
- .c file containing the C-code
- .pxd file containing the function declaration in a python readable way
- .pyx file containing the wrapper function calling the C-function
- a setup.py file needed to compile the C and cython code

## Example

We chose a simple example to explain the structure and the content of the files needed. Our example finds the nearest element to a given key. We chose such an example to have a function expecting an array and some integer to emphasize how to pass a python array to a C-function.

The example files are named:
- minimal_to_wrap.h
- minimal_to_wrap.c
- cython_declaration_module.pxd
- find_closest_element.pyx
- setup.py


### C-header file - minimal_to_wrap.h

This is the normal C header file containing the function declaration.

In [ ]:
//header file with function devlaration

int find_closest_element_in_c(double * input_data, int size_of_data, double key);

### C-file - minimal_to_wrap.c

This is the C-code with the function definition to include in the python code. Our example function finds the closest element to a given key.

The function ``find_closest_element_in_c`` takes to arguments; an input array ``input_data``, a array length ``size_of_data`` and a key ``key``. It then returns the index of the closest "closest" element to the given key.

In [ ]:
#include <math.h>

//Returns the index of the element of input_data
//closest to key
int find_closest_element_in_c(double * input_data, int size_of_data, double key)
{
    if (size_of_data <= 0)
        return -1;
    //Linearly search through the array to find the closest element
    //distance stores the current minimal distance
    //closest stores the index
    //Sorting the array first would certainly be much faster!
    double distance = fabs(input_data[0] - key);
    int closest = 0;
    for(int i = 0; i < size_of_data; ++i)
    {
        if (fabs(input_data[i] - key) < distance )
        {
            closest = i;
            distance = fabs(input_data[i] - key);
        }
    }
    return closest;
}

### .pxd-file - cython_declaration_module.pxd
This is the cython declaration of the C-function to include into the python code

The first line is the docstring, then follows the cython definition of the function ``cdef extern from ".h-file"`` which basically just specifies the external file where to find the C-function definition. 
The next line then repeats the "normal" C-function definition. 

In [ ]:
""" Cython declaration of the C extension. 
"""

cdef extern from "minimal_to_wrap.h":
	int find_closest_element_in_c(double * input_data, int size_of_data, double key);

### .pyx-file - find_closest_element.pyx

This is the actual cython code wrapping around the C-function. 

Because we want to use cython we need to use ``cimport numpy as c_np`` and ``cimport cython_declaration_module``.

We then define our python function ``find_closest_element_wrapper`` which takes two input parameters, an input array ``input_data`` and a ``key``. Again, because we use cython, we need to specify the type of the input array. Here we use a numpy ndarray of type ``float64_t``.

Inside our python function we call the C-function via ``cython_declaration_module.find_closest_element_in_c(&input_data[0], len(input_data), key)``
where we need to pass the first argument in the manner we would pass arrays in C. This means we pass the pointer to the first element of our array ``&input_data[0]``. The second argument is the length of the array, as demanded by the C definition of the function. The third argument then is just passing the key.

As a last step we return the index of the closest element as the result calculated by the C-function.


In [ ]:
#import numpy as np
cimport numpy as c_np 

cimport cython_declaration_module

def find_closest_element_wrapper(c_np.ndarray[c_np.float64_t, ndim=1] input_data, key):
    cdef int result
    result = cython_declaration_module.find_closest_element_in_c(&input_data[0], len(input_data), key)
    return result

### setup.py file - find_closest_element.pyx

This is the esential file to make the C-function callable from plain python. This file is needed to compile the C/cython code in order to create a importable library (.so file). 

To make the setup file run, we need to import some essential modules.

``import numpy as np``  
``from distutils.extension import Extension``
`` from distutils.extension import Extension``  
``from distutils.core import setup``  
``from Cython.Build import cythonize``

We then define the source files:
``sources = ['find_closest_element.pyx', 'minimal_to_wrap.c']``

!!! Important: The name given to the .pyx file must match the name specified in the Extension section!

The next line  
``extension_obj_instance = Extension(name="find_closest_element", sources=sources, include_dirs=[np.get_include()])``
builds the extension object (note the use of the include_dirs keyword argument - necessary whenever our cython wrapper requires a cimport of numpy)

Finally we define what to do when setup.py is called.
``setup(name="cython_wrapper",ext_modules = cythonize([extension_obj_instance]))``


In [ ]:
import numpy as np
from distutils.extension import Extension
from distutils.core import setup
from Cython.Build import cythonize

sources = ['find_closest_element.pyx', 'minimal_to_wrap.c']

extension_obj_instance = Extension(name="find_closest_element", sources=sources,
    include_dirs=[np.get_include()])

setup(name="cython_wrapper",ext_modules = cythonize([extension_obj_instance]))

### Inclusion into python

short example program for importing and calling the newly defined C-function

In [12]:
import os
os.system("python setup.py build_ext --inplace")

import numpy as np
from find_closest_element import find_closest_element_wrapper

data = np.random.rand(1000)*1000
key = 34

result = find_closest_element_wrapper(data, key)
print data[result]

34.7337614986
